In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
import os
print(os.getcwd())

D:\멋사_데이터분석_부트캠프\데이터톤 프로젝트\jupyter


In [6]:
fa_df = pd.read_csv('../KBO FA 자격 선수 명단 (2013 ~ 2024).csv', encoding='cp949')

In [7]:
fa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연도      291 non-null    int64 
 1   구단명     291 non-null    object
 2   선수명     291 non-null    object
 3   생년월일    291 non-null    object
 4   년       291 non-null    int64 
 5   월       291 non-null    int64 
 6   일       291 non-null    int64 
 7   포지션     291 non-null    object
 8   인정년수    291 non-null    int64 
 9   구분      291 non-null    object
 10  비고      291 non-null    object
 11  FA 등급   118 non-null    object
dtypes: int64(5), object(7)
memory usage: 27.4+ KB


In [8]:
fa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연도      291 non-null    int64 
 1   구단명     291 non-null    object
 2   선수명     291 non-null    object
 3   생년월일    291 non-null    object
 4   년       291 non-null    int64 
 5   월       291 non-null    int64 
 6   일       291 non-null    int64 
 7   포지션     291 non-null    object
 8   인정년수    291 non-null    int64 
 9   구분      291 non-null    object
 10  비고      291 non-null    object
 11  FA 등급   118 non-null    object
dtypes: int64(5), object(7)
memory usage: 27.4+ KB


In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from urllib.parse import quote, urlparse, parse_qs  # urlparse와 parse_qs 추가
import time
import random
import sys

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# 일반적인 브라우저의 user-agent 설정
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
driver = webdriver.Chrome(options=chrome_options)

df2 = fa_df.copy()
df2 = df2.drop_duplicates(subset=['선수명', '생년월일'])
players = df2.to_dict(orient='records')

results = []

for player in players:
    name = player['선수명']
    birth_csv = player['생년월일'].strip()
    encoded_name = quote(name)
    print(f"encoded_name : {encoded_name}")
    
    search_url = f"https://statiz.sporki.com/player/?m=search&s={encoded_name}"
    print(f"search_url : {search_url}")
    
    driver.get(search_url)
    time.sleep(random.uniform(2, 5))
    current_url = driver.current_url
    print(f"current_url : {current_url}")
    
    page_source = driver.page_source
    
    if "p_no=" in current_url:
        p_no = parse_qs(urlparse(current_url).query)['p_no'][0]  # 이제 불러온 함수를 직접 사용
    print(p_no)


    # try:
    #     if "p_no=" in current_url:
    #         # 리다이렉션 된 경우: 바로 처리
    #         p_no = parse_qs(urlparse(current_url).query)['p_no'][0]

    #     else:
    #         # 동명이인 리스트 테이블 → 생년월일로 클릭
    #         rows = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table tbody tr")))
    #         p_no = None
    #         for row in rows:
    #             cols = row.find_elements(By.TAG_NAME, 'td')
    #             if len(cols) >= 3:
    #                 name_in_table = cols[1].text.strip()
    #                 birth_in_table = cols[2].text.strip()

    #                 if name_in_table == name and birth_in_table == birth_csv:
    #                     link = cols[1].find_element(By.TAG_NAME, 'a')
    #                     link.click()
    #                     time.sleep(1.5)
    #                     new_url = driver.current_url
    #                     p_no = parse_qs(urlparse(new_url).query)['p_no'][0]
    #                     break

    #         if not p_no:
    #             print(f"[매칭 실패] {name} ({birth_csv}) : 정확한 선수 못 찾음")
    #             continue

    #     # 연도별 지표 페이지 접속
    #     year_url = f"https://statiz.sporki.com/player/?m=year&p_no={p_no}"
    #     driver.get(year_url)
    #     time.sleep(1.5)

    #     # 테이블 추출
    #     table = driver.find_element(By.TAG_NAME, 'table')
    #     rows = table.find_elements(By.TAG_NAME, 'tr')

    #     data = []
    #     for i, row in enumerate(rows):
    #         cols = row.find_elements(By.TAG_NAME, 'th' if i == 0 else 'td')
    #         data.append([col.text.strip() for col in cols])

    #     # 저장
    #     df_table = pd.DataFrame(data[1:], columns=data[0])
    #     safe_name = name.replace(" ", "_")
    #     filename = os.path.join(save_dir, f"{safe_name}.csv")
    #     df_table.to_csv(filename, index=False, encoding='utf-8-sig')
    #     print(f"[저장 완료] {filename}")

    # except Exception as e:
    #     print(f"[오류] {name}: {e}")

driver.quit()

encoded_name : %EA%B9%80%EC%9B%90%EC%84%AD
search_url : https://statiz.sporki.com/player/?m=search&s=%EA%B9%80%EC%9B%90%EC%84%AD
current_url : https://statiz.sporki.com/player/?m=playerinfo&p_no=10029
10029
encoded_name : %EC%9C%A0%EB%8F%99%ED%9B%88
search_url : https://statiz.sporki.com/player/?m=search&s=%EC%9C%A0%EB%8F%99%ED%9B%88
current_url : https://statiz.sporki.com/player/?m=playerinfo&p_no=10060
10060
encoded_name : %EC%9D%B4%ED%98%84%EA%B3%A4
search_url : https://statiz.sporki.com/player/?m=search&s=%EC%9D%B4%ED%98%84%EA%B3%A4
current_url : https://statiz.sporki.com/player/?m=playerinfo&p_no=10022
10022
encoded_name : %EA%B9%80%EC%9D%BC%EA%B2%BD
search_url : https://statiz.sporki.com/player/?m=search&s=%EA%B9%80%EC%9D%BC%EA%B2%BD


KeyboardInterrupt: 